<a href="https://colab.research.google.com/github/jasongan1028-cmd/Halo/blob/main/Milestone_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Milestone 1: A Spatiotemporal Analysis of Socioeconomic and Demographic Correlates in Walmart Supercenter Closures (2019-2024)**


Team Members: Jason Gan (jasongan), Tingting Zhu (zhutt), Xiaojun Xu (xiaojunx) \
Mentor: Erik Lang

# **Set Up Environment**

In [ ]:
pip install requests

In [1]:
import requests
import pandas as pd
import time
import json
import os
import random

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
!ls /content/drive/MyDrive/walmart_OSM

'B01003 Total population.csv'		     walmart_2024_geocode.csv
'B03002 Hispanic or Latino population.csv'   walmart_acs_master.csv
'B19013 Median household income.csv'	     walmart_closure_comparison.csv
 raw_2024_by_state			     walmart_closure_with_acs.csv
 walmart_2024_all_states.csv		     walmart_Kaggle.csv
 walmart_2024_all_states.json		    'walmart_Kaggle_final .csv'
 walmart_2024_final.csv			     walmart_Kaggle_final.csv


# **Primary Data: Walmart Closures During 2019-2024**

## **1. Data Extraction: Open Street Map API (2024)**

In [ ]:
|# check skip-retry
OUT_DIR = "/content/drive/MyDrive/walmart_OSM/raw_2024_by_state"

US_STATES = {
    "Alabama": "US-AL", "Alaska": "US-AK", "Arizona": "US-AZ", "Arkansas": "US-AR",
    "California": "US-CA", "Colorado": "US-CO", "Connecticut": "US-CT", "Delaware": "US-DE",
    "District of Columbia": "US-DC","Florida": "US-FL","Georgia": "US-GA", "Hawaii": "US-HI",
    "Idaho": "US-ID", "Illinois": "US-IL", "Indiana": "US-IN", "Iowa": "US-IA",
    "Kansas": "US-KS", "Kentucky": "US-KY", "Louisiana": "US-LA", "Maine": "US-ME",
    "Maryland": "US-MD", "Massachusetts": "US-MA", "Michigan": "US-MI", "Minnesota": "US-MN",
    "Mississippi": "US-MS", "Missouri": "US-MO", "Montana": "US-MT", "Nebraska": "US-NE",
    "Nevada": "US-NV", "New Hampshire": "US-NH", "New Jersey": "US-NJ", "New Mexico": "US-NM",
    "New York": "US-NY", "North Carolina": "US-NC", "North Dakota": "US-ND", "Ohio": "US-OH",
    "Oklahoma": "US-OK", "Oregon": "US-OR", "Pennsylvania": "US-PA", "Rhode Island": "US-RI",
    "South Carolina": "US-SC", "South Dakota": "US-SD", "Tennessee": "US-TN", "Texas": "US-TX",
    "Utah": "US-UT", "Vermont": "US-VT", "Virginia": "US-VA", "Washington": "US-WA",
    "West Virginia": "US-WV", "Wisconsin": "US-WI", "Wyoming": "US-WY"
}

def state_outfile(iso_code):
    return os.path.join(OUT_DIR, f"walmart_{iso_code}_2024.json")


def check_file(path):
    if not os.path.exists(path):
        return "MISSING"

    try:
        size = os.path.getsize(path)
        if size == 0:
            return "EMPTY FILE"

        with open(path, "r") as f:
            data = json.load(f)

        if not isinstance(data, list):
            return "NOT A LIST"

        if len(data) == 0:
            return "EMPTY LIST"

        return f"OK ({len(data)} items)"

    except Exception as e:
        return f"CORRUPTED JSON: {e}"


print("==== Pre-check existing JSON files ====\n")

for state_name, iso_code in US_STATES.items():
    path = state_outfile(iso_code)
    status = check_file(path)

    if status.startswith("OK"):
        action = "→ WILL SKIP"
    else:
        action = "→ WILL RE-FETCH"

    print(f"{state_name:20s} | {status:35s} {action}")


==== Pre-check existing JSON files ====

Alabama              | OK (199 items)                      → WILL SKIP
Alaska               | OK (11 items)                       → WILL SKIP
Arizona              | OK (219 items)                      → WILL SKIP
Arkansas             | OK (184 items)                      → WILL SKIP
California           | OK (431 items)                      → WILL SKIP
Colorado             | OK (179 items)                      → WILL SKIP
Connecticut          | OK (42 items)                       → WILL SKIP
Delaware             | OK (10 items)                       → WILL SKIP
District of Columbia | OK (2 items)                        → WILL SKIP
Florida              | OK (515 items)                      → WILL SKIP
Georgia              | OK (296 items)                      → WILL SKIP
Hawaii               | OK (12 items)                       → WILL SKIP
Idaho                | OK (53 items)                       → WILL SKIP
Illinois             | OK (237 items

In [ ]:
# Code3: FL（split to 4 bbox)
import os
import json
import time
import random
import requests

# -----------------------
# Config
# -----------------------
ISO_CODE = "US-FL"
STATE_NAME = "Florida"
DATE_ISO = "2024-12-31T23:59:59Z"

OVERPASS_URLS = [
    "https://overpass-api.de/api/interpreter",
    "https://overpass.kumi.systems/api/interpreter",
]

HEADERS = {"User-Agent": "walmart-research/1.0 (contact: zhutt@umich.edu)"}

OUT_DIR = "/content/drive/MyDrive/walmart_OSM/raw_2024_by_state"
os.makedirs(OUT_DIR, exist_ok=True)
OUT_PATH = os.path.join(OUT_DIR, f"walmart_{ISO_CODE}_2024.json")

# Florida rough bounding box, split into 2x2 (4 queries)
# (minlat, minlon, maxlat, maxlon)
FL_BOXES = [
    (24.4, -87.7, 27.75, -83.85),
    (24.4, -83.85, 27.75, -80.0),
    (27.75, -87.7, 31.1, -83.85),
    (27.75, -83.85, 31.1, -80.0),
]

# -----------------------
# Helpers
# -----------------------
def atomic_write_json(path: str, data):
    tmp = path + ".tmp"
    with open(tmp, "w") as f:
        json.dump(data, f, ensure_ascii=False)
    os.replace(tmp, path)

def dedup_elements(elems):
    seen = set()
    out = []
    for e in elems:
        k = (e.get("type"), e.get("id"))
        if k in seen:
            continue
        seen.add(k)
        out.append(e)
    return out

def build_query_bbox(minlat, minlon, maxlat, maxlon, date_iso=DATE_ISO):
    # Your original matching logic, applied to bbox instead of state area
    return f"""
    [out:json][timeout:1800][date:"{date_iso}"];
    (
      nwr({minlat},{minlon},{maxlat},{maxlon})["brand"="Walmart"];
      nwr({minlat},{minlon},{maxlat},{maxlon})["name"~"^Walmart",i];
    );
    out center tags;
    """

def run_overpass(query: str, max_tries: int = 10):
    last_err = None
    for attempt in range(max_tries):
        url = random.choice(OVERPASS_URLS)
        try:
            r = requests.post(
                url,
                data={"data": query},
                headers=HEADERS,
                timeout=(30, 1900)  # connect, read
            )

            if r.status_code == 200:
                try:
                    return r.json().get("elements", [])
                except Exception as e:
                    last_err = f"JSON decode error @ {url}: {repr(e)}"

            elif r.status_code in (429, 502, 503, 504):
                last_err = f"HTTP {r.status_code} @ {url}"

            else:
                last_err = f"HTTP {r.status_code} @ {url}: {r.text[:200]}"
                print("[NON-RETRY]", last_err)
                return None

        except Exception as e:
            last_err = f"EXC @ {url}: {repr(e)}"

        sleep_s = min(240, (2 ** attempt) + random.random() * 10)
        print(f"[RETRY] {last_err} | sleep {sleep_s:.1f}s")
        time.sleep(sleep_s)

    print(f"[FAIL] After {max_tries} tries. Last error: {last_err}")
    return None

# -----------------------
# Fetch Florida by bbox blocks
# -----------------------
all_elems = []
for i, (minlat, minlon, maxlat, maxlon) in enumerate(FL_BOXES, start=1):
    print(f"Fetching FL block {i}/{len(FL_BOXES)} bbox=({minlat},{minlon},{maxlat},{maxlon})")
    q = build_query_bbox(minlat, minlon, maxlat, maxlon)
    elems = run_overpass(q, max_tries=12)

    if elems is None:
        raise RuntimeError(f"Florida block {i} failed. Try increasing split (3x3) or removing [date:].")

    # add state tag
    for item in elems:
        if isinstance(item, dict):
            tags = item.get("tags")
            if isinstance(tags, dict):
                tags["state_ref"] = STATE_NAME

    all_elems.extend(elems)
    print(f"  Block {i} got {len(elems)} elements.")
    time.sleep(6)  # be nice

# Dedup + Save
all_elems = dedup_elements(all_elems)
print(f"\nFlorida total after dedup: {len(all_elems)}")

atomic_write_json(OUT_PATH, all_elems)
print(f"Saved: {OUT_PATH}")


Fetching FL block 1/4 bbox=(24.4,-87.7,27.75,-83.85)
  Block 1 got 0 elements.
Fetching FL block 2/4 bbox=(24.4,-83.85,27.75,-80.0)
  Block 2 got 177 elements.
Fetching FL block 3/4 bbox=(27.75,-87.7,31.1,-83.85)
  Block 3 got 54 elements.
Fetching FL block 4/4 bbox=(27.75,-83.85,31.1,-80.0)
  Block 4 got 284 elements.

Florida total after dedup: 515
Saved: /content/drive/MyDrive/walmart_OSM/raw_2024_by_state/walmart_US-FL_2024.json


**Problems:** National-scale queries for Walmart locations consistently triggered API timeouts and memory limits on the Overpass servers, while network instability risked corrupting data saved directly to cloud storage.

**Solutions:** We engineered a partitioned extraction pipeline that iterates by state, utilizing Exponential Backoff for retries and Atomic Writing for file saving. Segmenting the data ensures we bypass server-side "kills," while atomic operations and checkpointing allow the script to resume safely after interruptions without losing progress or corrupting the existing dataset.

**Expectations:** A complete, verified snapshot of 2024 Walmart locations across 51 jurisdictions, structured for a precise longitudinal merge with 2019 baseline data.

In [ ]:
# Portions of this code were generated with the assistance of OpenAI GPT-4

# Summary of Key Prompts: To optimize the Walmart 2024 extraction script, we focused on three critical failure points:
# 1. Iterative Reliability: "How can we redesign the script to ensure it successfully iterates through all 50 states without timing out or being blocked by the server?"
# 2. Resilience Strategy: "How can we handle HTTP 429 and 504 errors from the Overpass API?"
# 3. Data Integrity: "How can we prevent data corruption when saving progress to a cloud-synced directory?"

# Generated on: February 2026

US_STATES = {
    "Alabama": "US-AL", "Alaska": "US-AK", "Arizona": "US-AZ", "Arkansas": "US-AR",
    "California": "US-CA", "Colorado": "US-CO", "Connecticut": "US-CT", "Delaware": "US-DE",
    "District of Columbia": "US-DC", "Florida": "US-FL", "Georgia": "US-GA", "Hawaii": "US-HI",
    "Idaho": "US-ID", "Illinois": "US-IL", "Indiana": "US-IN", "Iowa": "US-IA",
    "Kansas": "US-KS", "Kentucky": "US-KY", "Louisiana": "US-LA", "Maine": "US-ME",
    "Maryland": "US-MD", "Massachusetts": "US-MA", "Michigan": "US-MI", "Minnesota": "US-MN",
    "Mississippi": "US-MS", "Missouri": "US-MO", "Montana": "US-MT", "Nebraska": "US-NE",
    "Nevada": "US-NV", "New Hampshire": "US-NH", "New Jersey": "US-NJ", "New Mexico": "US-NM",
    "New York": "US-NY", "North Carolina": "US-NC", "North Dakota": "US-ND", "Ohio": "US-OH",
    "Oklahoma": "US-OK", "Oregon": "US-OR", "Pennsylvania": "US-PA", "Rhode Island": "US-RI",
    "South Carolina": "US-SC", "South Dakota": "US-SD", "Tennessee": "US-TN", "Texas": "US-TX",
    "Utah": "US-UT", "Vermont": "US-VT", "Virginia": "US-VA", "Washington": "US-WA",
    "West Virginia": "US-WV", "Wisconsin": "US-WI", "Wyoming": "US-WY"
}

OVERPASS_URLS = [
  "https://overpass-api.de/api/interpreter",
  "https://overpass.kumi.systems/api/interpreter",
]

HEADERS = {"User-Agent": "walmart-research/1.0 (contact: zhutt@umich.edu)"}

OUT_DIR = "/content/drive/MyDrive/walmart_OSM/raw_2024_by_state"
os.makedirs(OUT_DIR, exist_ok=True)

def state_outfile(iso_code: str) -> str:
    return os.path.join(OUT_DIR, f"walmart_{iso_code}_2024.json")

def build_query(state_name: str, iso_code: str, date_iso: str) -> str:
    return f"""
    [out:json][timeout:1800][date:"{date_iso}"];
    rel["name"="{state_name}"]["boundary"="administrative"]["admin_level"="4"]["ISO3166-2"="{iso_code}"];
    map_to_area->.st;
    (
      nwr["brand"="Walmart"](area.st);
      nwr["name"~"^Walmart",i](area.st);
    );
    out center tags;
    """

def load_state_file_if_ok(out_path: str):
    """Return list if file exists and is a non-empty list; else None."""
    if not os.path.exists(out_path):
        return None
    try:
        with open(out_path, "r") as f:
            elems = json.load(f)
        if isinstance(elems, list) and len(elems) > 0:
            return elems
        return None
    except Exception:
        return None

def atomic_write_json(path: str, data):
    """Write JSON safely: write to tmp then replace."""
    tmp = path + ".tmp"
    with open(tmp, "w") as f:
        json.dump(data, f, ensure_ascii=False)
    os.replace(tmp, path)

def get_walmarts_by_state(state_name: str, iso_code: str,
                         date_iso: str = "2024-12-31T23:59:59Z",
                         max_tries: int = 10):
    query = build_query(state_name, iso_code, date_iso)
    last_err = None

    for attempt in range(max_tries):
        url = random.choice(OVERPASS_URLS)

        try:
            r = requests.post(
                url,
                data={"data": query},
                headers=HEADERS,
                timeout=(30, 1900)  # connect timeout, read timeout
            )

            if r.status_code == 200:
                try:
                    return r.json().get("elements", [])
                except Exception as e:
                    last_err = f"JSON decode error @ {url}: {repr(e)}"
                    sleep_s = min(180, (2 ** attempt) + random.random() * 5)
                    print(f"  {state_name}: {last_err}, retry in {sleep_s:.1f}s")
                    time.sleep(sleep_s)
                    continue

            if r.status_code in (429, 502, 503, 504):
                last_err = f"HTTP {r.status_code} @ {url}"
                sleep_s = min(180, (2 ** attempt) + random.random() * 5)
                print(f"  {state_name}: {last_err}, retry in {sleep_s:.1f}s")
                time.sleep(sleep_s)
                continue

            last_err = f"HTTP {r.status_code} @ {url}: {r.text[:120]}"
            print(f"  {state_name}: {last_err}")
            return None

        except Exception as e:
            last_err = f"EXC @ {url}: {repr(e)}"
            sleep_s = min(180, (2 ** attempt) + random.random() * 5)
            print(f"  {state_name}: {last_err}, retry in {sleep_s:.1f}s")
            time.sleep(sleep_s)

    print(f"[FAIL] {state_name} after {max_tries} tries. Last error: {last_err}")
    return None


# FETCH with checkpointing

failed = []
total_saved = 0
skipped = 0
retried = 0

for state_name, iso_code in US_STATES.items():
    out_path = state_outfile(iso_code)

    existing = load_state_file_if_ok(out_path)
    if existing is not None:
        print(f"Skip {state_name} (exists): {len(existing)}")
        total_saved += len(existing)
        skipped += 1
        continue
    else:
        if os.path.exists(out_path):
            print(f"Re-try {state_name} (file exists but empty/unreadable).")
            retried += 1

    print(f"Fetching data for {state_name}...")
    elems = get_walmarts_by_state(state_name, iso_code)

    if elems is None:
        failed.append(state_name)
        continue

    # Add state reference tag
    for item in elems:
        if isinstance(item, dict) and "tags" in item and isinstance(item["tags"], dict):
            item["tags"]["state_ref"] = state_name

    # Save
    atomic_write_json(out_path, elems)

    print(f"  Done. Found {len(elems)} items.")
    total_saved += len(elems)

    time.sleep(5)

print("\nFETCH PHASE DONE.")
print("Skipped states:", skipped)
print("Retried states:", retried)
print("Total saved (counted from files + new fetch):", total_saved)
print("FAILED states:", failed)


Skip Alabama (exists): 199
Skip Alaska (exists): 11
Skip Arizona (exists): 219
Skip Arkansas (exists): 184
Skip California (exists): 431
Skip Colorado (exists): 179
Skip Connecticut (exists): 42
Skip Delaware (exists): 10
Skip District of Columbia (exists): 2
Skip Florida (exists): 515
Skip Georgia (exists): 296
Skip Hawaii (exists): 12
Skip Idaho (exists): 53
Skip Illinois (exists): 237
Skip Indiana (exists): 160
Skip Iowa (exists): 82
Skip Kansas (exists): 110
Skip Kentucky (exists): 130
Skip Louisiana (exists): 180
Skip Maine (exists): 43
Skip Maryland (exists): 68
Skip Massachusetts (exists): 65
Skip Michigan (exists): 121
Skip Minnesota (exists): 101
Skip Mississippi (exists): 114
Skip Missouri (exists): 202
Skip Montana (exists): 36
Skip Nebraska (exists): 72
Skip Nevada (exists): 69
Skip New Hampshire (exists): 47
Skip New Jersey (exists): 76
Skip New Mexico (exists): 77
Skip New York (exists): 146
Skip North Carolina (exists): 274
Skip North Dakota (exists): 16
Skip Ohio (exist

In [ ]:
# merge 51 state
import os
import json
import glob

IN_DIR = "/content/drive/MyDrive/walmart_OSM/raw_2024_by_state"
OUT_PATH = "/content/drive/MyDrive/walmart_OSM/walmart_2024_all_states.json"

all_walmarts = []

paths = sorted(glob.glob(os.path.join(IN_DIR, "walmart_US-*_2024.json")))
print("Found files:", len(paths))

for p in paths:
    with open(p, "r") as f:
        data = json.load(f)
    print(f"Reading {os.path.basename(p)} | {len(data)} items")
    all_walmarts.extend(data)

print("\nTotal after simple merge:", len(all_walmarts))

with open(OUT_PATH, "w") as f:
    json.dump(all_walmarts, f)

print("Saved merged JSON to:", OUT_PATH)


Found files: 51
Reading walmart_US-AK_2024.json | 11 items
Reading walmart_US-AL_2024.json | 199 items
Reading walmart_US-AR_2024.json | 184 items
Reading walmart_US-AZ_2024.json | 219 items
Reading walmart_US-CA_2024.json | 431 items
Reading walmart_US-CO_2024.json | 179 items
Reading walmart_US-CT_2024.json | 42 items
Reading walmart_US-DC_2024.json | 2 items
Reading walmart_US-DE_2024.json | 10 items
Reading walmart_US-FL_2024.json | 515 items
Reading walmart_US-GA_2024.json | 296 items
Reading walmart_US-HI_2024.json | 12 items
Reading walmart_US-IA_2024.json | 82 items
Reading walmart_US-ID_2024.json | 53 items
Reading walmart_US-IL_2024.json | 237 items
Reading walmart_US-IN_2024.json | 160 items
Reading walmart_US-KS_2024.json | 110 items
Reading walmart_US-KY_2024.json | 130 items
Reading walmart_US-LA_2024.json | 180 items
Reading walmart_US-MA_2024.json | 65 items
Reading walmart_US-MD_2024.json | 68 items
Reading walmart_US-ME_2024.json | 43 items
Reading walmart_US-MI_2024.

**2019 Dataset:** Originally, an OSM 2019 temporal query was attempted with the same approach. However, due to the crowdsourced nature of OpenStreetMap, historical data density in 2019 was found to be insufficient for a reliable longitudinal study.

**Replacement Source**: A high-fidelity Kaggle Walmart locations Dataset (as of November 2018) was utilized as the baseline for the 2019-era comparison.

## **2. Data Cleaning (2024)**

### **Data Transformation:** Flattening Hierarchical OSM JSON into a Tabular Store Panel

In [ ]:
# Load the raw JSON data

with open('/content/drive/MyDrive/walmart_OSM/walmart_2024_all_states.json', 'r') as f:
    data = json.load(f)

rows = []

for item in data:
    t = item.get('tags', {})

    if item.get('type') == 'node':
        lat = item.get('lat')
        lon = item.get('lon')
    else:
        center = item.get('center', {})
        lat = center.get('lat')
        lon = center.get('lon')

    rows.append({
        'osm_id': item.get('id'),
        'osm_type': item.get('type'),

        'lat': lat,
        'lon': lon,

        'name': t.get('name'),
        'brand': t.get('brand'),
        'operator': t.get('operator'),

        'street': t.get('addr:street'),
        'city': t.get('addr:city'),
        'state': t.get('addr:state'),
        'zip': t.get('addr:postcode'),

        'state_ref': t.get('state_ref'),
    })

df_2024 = pd.DataFrame(rows)

print("Total rows:", len(df_2024))
print("Missing coords:", df_2024['lat'].isna().sum())
print(df_2024.head())

df_2024.to_csv('/content/drive/MyDrive/walmart_OSM/walmart_2024_all_states.csv', index=False)


Total rows: 7349
Missing coords: 0
        osm_id osm_type        lat         lon  \
0  12097718365     node  61.192524 -149.880676   
1  12097718366     node  61.191960 -149.880678   
2     28615011      way  61.192314 -149.879649   
3    170069699      way  55.375440 -131.720547   
4    209450680      way  57.811955 -152.365307   

                                 name    brand operator         street  \
0            Walmart Grocery Entrance     None     None           None   
1  Walmart Home and Pharmacy Entrance     None     None           None   
2                 Walmart Supercenter  Walmart  Walmart       A Street   
3                             Walmart  Walmart  Walmart  Don King Road   
4                             Walmart  Walmart  Walmart  Mill Bay Road   

        city state    zip state_ref  
0       None  None   None    Alaska  
1       None  None   None    Alaska  
2  Anchorage    AK  99503    Alaska  
3  Ketchikan    AK  99901    Alaska  
4     Kodiak    AK  99615    

### **Data Enrichment:** Geospatial Gap Filling via Reverse Geocoding

**Problems**: The initial OpenStreetMap (OSM) extraction contained inconsistent address metadata; several store records lacked critical geographic identifiers (City, State, or ZIP code), which are required to join the store data with Census ZCTA demographics.

**Solutions:** We implemented a Reverse Geocoding pipeline using the Nominatim engine and the geopy library, targeting only the subset of records with missing address values. To ensure ethical API usage and pipeline stability, we utilized a RateLimiter to enforce a mandatory 1-second delay and integrated a checkpointing system that commits progress to a CSV every 200 records. This design prevents data loss and avoids redundant server requests in the event of a network timeout.

**Expectations**: A fully enriched dataset where all store locations are mapped to their respective municipalities and ZIP codes, enabling a high-precision spatial merge with the socioeconomic baseline.

In [ ]:
# Portions of this code were generated with the assistance of OpenAI GPT-4

# Summary of Key Prompts: To build a reliable enrichment pipeline, we used AI to address the following technical challenges:
# 1. "Write a script that performs reverse geocoding on rows where 'city', 'state', or 'zip' are missing."
# 2. "The Nominatim API has strict usage limits. How can I implement a RateLimiter that waits between requests and handles potential server timeouts or exceptions?"
# 3. "How can I add a checkpoint system to this loop so that if the internet disconnects, I don't lose the geocoding progress?"

# Generated on: February 2026

import pandas as pd
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

df_2024 = pd.read_csv("/content/drive/MyDrive/walmart_OSM/walmart_2024_all_states.csv")

# pick id column name
id_col = "osm_id" if "osm_id" in df_2024.columns else ("id" if "id" in df_2024.columns else None)
if id_col is None:
    raise ValueError("Cannot find an id column (expected 'osm_id' or 'id').")

# setup nominatim
geolocator = Nominatim(user_agent="walmart_geocoder_research (contact: zhutt@umich.edu)")

# wrapper
reverse = RateLimiter(
    geolocator.reverse,
    min_delay_seconds=1,
    max_retries=2,
    error_wait_seconds=5,
    swallow_exceptions=False
)

# only geocode missing rows
need = df_2024["city"].isna() | df_2024["state"].isna() | df_2024["zip"].isna()
to_fill_idx = df_2024.index[need].tolist()
print("Rows needing reverse geocode:", len(to_fill_idx), "out of", len(df_2024))

def fill_one(idx):
    row = df_2024.loc[idx]
    if pd.isna(row["lat"]) or pd.isna(row["lon"]):
        return  # can't reverse geocode without coords

    try:
        loc = reverse((row["lat"], row["lon"]), timeout=10)
        if not loc:
            return
        addr = (loc.raw or {}).get("address", {})

        # fill city
        if pd.isna(row["city"]):
            df_2024.at[idx, "city"] = addr.get("city") or addr.get("town") or addr.get("village") or addr.get("hamlet")

        # fill state
        if pd.isna(row["state"]):
            df_2024.at[idx, "state"] = addr.get("state_code") or addr.get("state")

        # fill zip
        if pd.isna(row["zip"]):
            df_2024.at[idx, "zip"] = addr.get("postcode")

    except Exception as e:
        print(f"Error at {id_col}={row.get(id_col)} idx={idx}: {e}")

# Checkpoint
OUT_PATH = "/content/drive/MyDrive/walmart_OSM/walmart_2024_geocode.csv"
SAVE_EVERY = 200

for n, idx in enumerate(to_fill_idx, 1):
    fill_one(idx)
    if n % SAVE_EVERY == 0:
        df_2024.to_csv(OUT_PATH, index=False)
        print(f"Checkpoint saved: {n}/{len(to_fill_idx)} -> {OUT_PATH}")

# final save
df_2024.to_csv(OUT_PATH, index=False)
print("Done. Saved:", OUT_PATH)


Rows needing reverse geocode: 2042 out of 7349
Checkpoint saved: 200/2042 -> /content/drive/MyDrive/walmart_OSM/walmart_2024_geocode.csv
Checkpoint saved: 400/2042 -> /content/drive/MyDrive/walmart_OSM/walmart_2024_geocode.csv
Checkpoint saved: 600/2042 -> /content/drive/MyDrive/walmart_OSM/walmart_2024_geocode.csv
Checkpoint saved: 800/2042 -> /content/drive/MyDrive/walmart_OSM/walmart_2024_geocode.csv
Checkpoint saved: 1000/2042 -> /content/drive/MyDrive/walmart_OSM/walmart_2024_geocode.csv
Checkpoint saved: 1200/2042 -> /content/drive/MyDrive/walmart_OSM/walmart_2024_geocode.csv
Checkpoint saved: 1400/2042 -> /content/drive/MyDrive/walmart_OSM/walmart_2024_geocode.csv
Checkpoint saved: 1600/2042 -> /content/drive/MyDrive/walmart_OSM/walmart_2024_geocode.csv
Checkpoint saved: 1800/2042 -> /content/drive/MyDrive/walmart_OSM/walmart_2024_geocode.csv
Checkpoint saved: 2000/2042 -> /content/drive/MyDrive/walmart_OSM/walmart_2024_geocode.csv
Done. Saved: /content/drive/MyDrive/walmart_OSM

### **Data Standardization:** Refinement and Structural Validation

**Problems:** The enriched dataset still contained "noisy" data, including non-Supercenter locations (like Fuel Station and Garden Cneter), non-standardized ZIP codes (e.g., ZIP+4 formats), and inconsistent state naming conventions (e.g., "Florida" vs. "FL").

**Solutions**:  We applied a strict structural normalization process that filtered for a uniform business entity, parsed geographic identifiers into a standard 5-digit format, and mapped state names to official two-letter ISO abbreviations  Standardizing these keys is vital for the longitudinal merge. Without this step, a store recorded as "FL" in 2019 and "Florida" in 2024 would appear as two different entities, or a store with a 9-digit ZIP code would fail to join with the 5-digit Census ZCTA data.

**Expectations:** A cleaned dataset of Walmart Supercenters with standardized geographic variables, ensuring an error-free integration with both the 2019 baseline and the ACS Census demographic layers.

In [ ]:
df_2024 = pd.read_csv("/content/drive/MyDrive/walmart_OSM/walmart_2024_geocode.csv")
# Keep only the requested columns
# cols_to_keep = ['id', 'name', 'zip', 'state']
# df_2024 = df_2024[cols_to_keep]

# Filter for Walmart Supercenter ONLY
target_names = ['Walmart Supercenter']
df_2024 = df_2024[df_2024['name'].isin(target_names)]

df_2024 = df_2024.dropna(subset=['zip'])

# Extract only the first ZIP code
# This splits by commas, spaces, or hyphens and takes the first part
df_2024['zip'] = df_2024['zip'].astype(str).str.split(r'[;\s\-]').str[0]

# Final safety check to remove any empty or invalid strings
df_2024 = df_2024[df_2024['zip'] != 'nan']
df_2024 = df_2024[df_2024['zip'].str.strip() != '']

df_2024 = df_2024[['lat', 'lon','name', 'city', 'state', 'zip']]

us_state_to_abbrev = {
    "ALABAMA":"AL","ALASKA":"AK","ARIZONA":"AZ","ARKANSAS":"AR","CALIFORNIA":"CA",
    "COLORADO":"CO","CONNECTICUT":"CT","DELAWARE":"DE","DISTRICT OF COLUMBIA":"DC",
    "FLORIDA":"FL","GEORGIA":"GA","HAWAII":"HI","IDAHO":"ID","ILLINOIS":"IL",
    "INDIANA":"IN","IOWA":"IA","KANSAS":"KS","KENTUCKY":"KY","LOUISIANA":"LA",
    "MAINE":"ME","MARYLAND":"MD","MASSACHUSETTS":"MA","MICHIGAN":"MI",
    "MINNESOTA":"MN","MISSISSIPPI":"MS","MISSOURI":"MO","MONTANA":"MT",
    "NEBRASKA":"NE","NEVADA":"NV","NEW HAMPSHIRE":"NH","NEW JERSEY":"NJ",
    "NEW MEXICO":"NM","NEW YORK":"NY","NORTH CAROLINA":"NC","NORTH DAKOTA":"ND",
    "OHIO":"OH","OKLAHOMA":"OK","OREGON":"OR","PENNSYLVANIA":"PA",
    "RHODE ISLAND":"RI","SOUTH CAROLINA":"SC","SOUTH DAKOTA":"SD",
    "TENNESSEE":"TN","TEXAS":"TX","UTAH":"UT","VERMONT":"VT","VIRGINIA":"VA",
    "WASHINGTON":"WA","WEST VIRGINIA":"WV","WISCONSIN":"WI","WYOMING":"WY",
    "PUERTO RICO":"PR"
}

def clean_state(df):
    df = df.copy()
    df["state"] = df["state"].str.upper().str.strip()
    df["state"] = df["state"].replace(us_state_to_abbrev)
    return df

df_2024 = clean_state(df_2024)

df_2024.to_csv('/content/drive/MyDrive/walmart_OSM/walmart_2024_final.csv', index=False)

print(df_2024.head())




         lat         lon                 name         city state    zip
2  61.192314 -149.879649  Walmart Supercenter    Anchorage    AK  99503
5  64.856394 -147.689449  Walmart Supercenter    Fairbanks    AK  99701
6  61.309108 -149.535586  Walmart Supercenter  Eagle River    AK  99577
7  60.564179 -151.225002  Walmart Supercenter        Kenai    AK  99611
8  61.568783 -149.365026  Walmart Supercenter      Wasilla    AK  99654


## **3. Data Cleaning (2019)**

In [ ]:
df_2019 = pd.read_csv("/content/drive/MyDrive/walmart_OSM/walmart_Kaggle.csv")
df_2019 = df_2019.rename(columns={'zip_code': 'zip', 'longitude':'lon', 'latitude': 'lat'})

df_2019 = df_2019[['name', 'city', 'state', 'zip', 'lat', 'lon']]
df_2019 = df_2019[df_2019['name'].str.contains('Supercenter', na=False)]

df_2019 = clean_state(df_2019)

df_2019.to_csv('/content/drive/MyDrive/walmart_OSM/walmart_Kaggle_final.csv', index=False)

## **4. Store Status Comparison by ZIP Code (2019-2024)**



### **Pre-Check**

In [ ]:
# pre-check
df_2019 = df_2019.copy()
df_2024 = df_2024.copy()

df_2019["state"] = df_2019["state"].str.upper().str.strip()
df_2024["state"] = df_2024["state"].str.upper().str.strip()

count_2019 = (
    df_2019.groupby("state")
    .size()
    .reset_index(name="stores_2019")
)

count_2024 = (
    df_2024.groupby("state")
    .size()
    .reset_index(name="stores_2024")
)

print(count_2019)
print(count_2024)

   state  stores_2019
0     AK            7
1     AL          101
2     AR           76
3     AZ           84
4     CA          141
5     CO           69
6     CT           12
7     DC            3
8     DE            6
9     FL          228
10    GA          154
11    IA           58
12    ID           23
13    IL          138
14    IN           97
15    KS           58
16    KY           78
17    LA           89
18    MA           26
19    MD           29
20    ME           19
21    MI           91
22    MN           65
23    MO          112
24    MS           65
25    MT           14
26    NC          142
27    ND           14
28    NE           35
29    NH           19
30    NJ           32
31    NM           35
32    NV           30
33    NY           80
34    OH          138
35    OK           81
36    OR           28
37    PA          116
38    PR           13
39    RI            5
40    SC           83
41    SD           15
42    TN          117
43    TX          387
44    UT  

### **Merge:** Spatial Join & Store Status Classification

In [ ]:
def normalize_keys(df):
    df = df.copy()

    df["city"]  = df["city"].astype("string").str.upper().str.strip()
    df["state"] = df["state"].astype("string").str.upper().str.strip()
    z = df["zip"].astype("string").str.strip()
    z = z.str.extract(r"(\d{5})", expand=False)
    df["zip"] = z.str.zfill(5)
    df = df[df["zip"].notna() & (df["zip"].str.len() == 5)].copy()

    return df

df_2019 = normalize_keys(df_2019)
df_2024 = normalize_keys(df_2024)

# merge
keys = ["zip", "city", "state"]

merged = df_2019.merge(
    df_2024,
    on=keys,
    how="outer",
    indicator=True,
    suffixes=("_2019", "_2024")
)

def classify_status(row):
    if row["_merge"] == "left_only":
        return "closed"
    elif row["_merge"] == "both":
        return "existing"
    else:
        return "newly_opened"

merged["store_status"] = merged.apply(classify_status, axis=1)

status_map = {
    "closed": 0,
    "existing": 1,
    "newly_opened": 2
}

merged["store_status_code"] = merged["store_status"].map(status_map)


print(merged["store_status"].value_counts())
print(merged["store_status_code"].value_counts())

merged.to_csv("/content/drive/MyDrive/walmart_OSM/walmart_closure_comparison.csv", index=False)


store_status
existing        3548
newly_opened     229
closed            88
Name: count, dtype: int64
store_status_code
1    3548
2     229
0      88
Name: count, dtype: int64


### **Post-Audit**

In [ ]:
print("zip not 5 digits:",
      (~merged["zip"].astype(str).str.match(r"^\d{5}$")).sum())

zip not 5 digits: 0


In [ ]:
# "zip+city+state" cause duplicate
keys = ["zip","city","state"]

dup19 = df_2019.duplicated(keys, keep=False).sum()
dup24 = df_2024.duplicated(keys, keep=False).sum()

print("2019 duplicate key rows:", dup19)
print("2024 duplicate key rows:", dup24)


2019 duplicate key rows: 177
2024 duplicate key rows: 259


In [ ]:
for label in ["closed","newly_opened"]:
    tmp = merged[merged["store_status"] == label]
    multi = tmp.groupby(keys).size().sort_values(ascending=False).head(10)
    print("\nTop duplicated keys in", label)
    print(multi)


Top duplicated keys in closed
zip    city              state
14043  LANCASTER         NY       1
14221  WILLIAMSVILLE     NY       1
14225  CHEEKTOWAGA       NY       1
14616  GREECE            NY       1
15005  ECONOMY           PA       1
15417  WEST BROWNSVILLE  PA       1
17319  ETTERS            PA       1
20001  WASHINGTON        DC       1
20176  LEESBURG          VA       1
21826  SALISBURY         MD       1
dtype: int64

Top duplicated keys in newly_opened
zip    city             state
74136  TULSA            OK       6
27858  EDWARDS ACRES    NC       3
62707  SPRINGFIELD      IL       3
01085  WESTFIELD        MA       2
35642  GULF SHORES      AL       2
32222  JACKSONVILLE     FL       2
33177  MIAMI            FL       2
33962  WINDING CYPRESS  FL       2
28584  SWANSBORO        NC       2
02767  RAYNHAM          MA       2
dtype: int64


# **Secondary Data: American Community Survey (ACS)**

## **1. Data Cleaning**

### **Data Standardization:** Median Income

In [ ]:
import pandas as pd

df_income = pd.read_csv("/content/drive/MyDrive/walmart_OSM/B19013 Median household income.csv",skiprows=[1])
df_income.columns = df_income.columns.str.lower()
df_income["zipcode"] = df_income["geo_id"].str[-5:]

df_income = df_income[["zipcode", "b19013_001e"]]
df_income = df_income.rename(columns={
    "b19013_001e": "median_income"
})

df_income["median_income"] = pd.to_numeric(df_income["median_income"], errors="coerce")


### **Data Standardization:** Total Population

In [ ]:
df_pop = pd.read_csv("/content/drive/MyDrive/walmart_OSM/B01003 Total population.csv",skiprows=[1])
df_pop.columns = df_pop.columns.str.lower()
df_pop["zipcode"] = df_pop["geo_id"].str[-5:]

df_pop = df_pop[["zipcode", "b01003_001e"]]
df_pop = df_pop.rename(columns={
    "b01003_001e": "total_population"
})

df_pop["total_population"] = pd.to_numeric(df_pop["total_population"], errors="coerce")

### **Data Standardization:** Hispanic Share

In [ ]:
df_race = pd.read_csv("/content/drive/MyDrive/walmart_OSM/B03002 Hispanic or Latino population.csv",skiprows=[1])
df_race.columns = df_race.columns.str.lower()
df_race["zipcode"] = df_race["geo_id"].str[-5:]
df_race = df_race[[
    "zipcode",
    "b03002_012e",  # Hispanic
    "b03002_001e"   # Total
]]

df_race["b03002_012e"] = pd.to_numeric(df_race["b03002_012e"], errors="coerce")
df_race["b03002_001e"] = pd.to_numeric(df_race["b03002_001e"], errors="coerce")

df_race["hispanic_share"] = (
    df_race["b03002_012e"] / df_race["b03002_001e"]
)

df_race = df_race[["zipcode", "hispanic_share"]]

### **Merge:** Multi-Variable Join of Census Attributes

In [ ]:
acs_master = (
    df_income
    .merge(df_pop, on="zipcode", how="left")
    .merge(df_race, on="zipcode", how="left")
)

acs_master = acs_master.dropna().reset_index(drop=True)

acs_master.to_csv(
    "/content/drive/MyDrive/walmart_OSM/walmart_acs_master.csv",
    index=False
)

## **2. Merge**: Spatial Aggregation and Socioeconomic Data Integration

### **Pre-Check**

In [ ]:
closure_path = "/content/drive/MyDrive/walmart_OSM/walmart_closure_comparison.csv"
acs_path     = "/content/drive/MyDrive/walmart_OSM/walmart_acs_master.csv"

df_closure = pd.read_csv(closure_path, dtype=str)
df_acs     = pd.read_csv(acs_path, dtype=str)

df_closure = df_closure.rename(columns={"zip": "zipcode"})

print("closure cols:", df_closure.columns.tolist())
print("acs cols:", df_acs.columns.tolist())

closure cols: ['name_2019', 'city', 'state', 'zipcode', 'lat_2019', 'lon_2019', 'lat_2024', 'lon_2024', 'name_2024', '_merge', 'store_status', 'store_status_code']
acs cols: ['zipcode', 'median_income', 'total_population', 'hispanic_share']


### **Merge**: ACS and Walmart Locations at ZIP-leve

In [ ]:
# closure to ZIP-level
zip_level = (
    df_closure
        .groupby(["zipcode", "city", "state"], as_index=False)
        .agg({
            "store_status_code": "min"   # closed(0) < existing(1) < newly_opened(2)
        })
)

zip_level["store_status"] = zip_level["store_status_code"].map({
    "0": "closed",
    "1": "existing",
    "2": "newly_opened"
})

#  ACS to ZIP-level
df_acs["zipcode"] = (
    df_acs["zipcode"]
      .astype(str)
      .str.extract(r"(\d{5})", expand=False)
      .str.zfill(5)
)

# merge
final = zip_level.merge(df_acs, on="zipcode", how="left")

print(final.head())
print("ACS match rate:", final["median_income"].notna().mean())

final.to_csv(
    "/content/drive/MyDrive/walmart_OSM/walmart_closure_with_acs.csv",
    index=False
)


  zipcode         city state store_status_code  store_status median_income  \
0   01020     CHICOPEE    MA                 2  newly_opened       70971.0   
1   01082         WARE    MA                 2  newly_opened       71023.0   
2   01085    WESTFIELD    MA                 2  newly_opened       83821.0   
3   01119  SPRINGFIELD    MA                 2  newly_opened       59172.0   
4   01247  NORTH ADAMS    MA                 2  newly_opened       55544.0   

  total_population        hispanic_share  
0            30230   0.20559047304002645  
1            10418   0.05442503359569975  
2            41466    0.0982732841364009  
3            14059    0.3316736610000711  
4            14967  0.054653571189951226  
ACS match rate: 0.9983074753173484


### **Post-Audit**

In [ ]:
print("closure rows:", len(df_closure))
print("acs rows:", len(df_acs))
print("closure zipcode missing:", final["zipcode"].isna().sum())
print("ACS matched rows:", final["median_income"].notna().sum())
print("ACS match rate:", f'{final["median_income"].notna().mean():.2%}')


closure rows: 3865
acs rows: 30509
closure zipcode missing: 0
ACS matched rows: 3539
ACS match rate: 99.83%
